# Element Parameter Detection

## Setup

In [1]:
import numpy as np
import math
import random
import os
import json
import torch
import copy
import sys

import datetime
import logging
import importlib
import shutil
import torch.nn as nn
from tqdm.notebook import tqdm_notebook as tqdm

import scipy.spatial.distance
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt

import ifcopenshell
from utils.JupyterIFCRenderer import JupyterIFCRenderer
from path import Path
import open3d as o3d

from src.elements import *
from src.ifc import *
from src. preparation import *
from src.dataset import *
from src.pointnet import *
from src.visualisation import *
from src.geometry import sq_distance

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
random.seed = 42

In [3]:
#path = Path("ModelNet10")
#path = Path('/content/drive/MyDrive/ElementNet/')
path = Path('output/')
#savepath = '/content/drive/MyDrive/ElementNet/'
savepath = 'models/'

In [ ]:
f = path/"tee/test/24102.pcd"
pointcloud = read_pcd(f)

In [ ]:
len(pointcloud)

## Transforms

In [ ]:
pcshow(*pointcloud.T)

### Normalize

In [ ]:
center_bbox(pointcloud)
dummy_properties = np.array([1.1, 2.2])
norm_pointcloud,_ = Normalize()((pointcloud, dummy_properties))
pcshow(*norm_pointcloud.T)

### Augmentations

random rotation and random noise 

In [ ]:
rot_pointcloud, _ = RandRotation_z()((norm_pointcloud, dummy_properties))
noisy_rot_pointcloud, _ = RandomNoise()((rot_pointcloud, dummy_properties))
pcshow(*noisy_rot_pointcloud.T)

## Dataset

Transforms for training.

In [4]:
train_transforms = transforms.Compose([
                    Normalize(),
#                    RandomNoise(),
                    ToTensor()
                    ])

In [5]:
cat= 'tee'
train_ds = PointCloudData(path, category=cat, transform=train_transforms)
valid_ds = PointCloudData(path, valid=True, folder='test', category=cat, transform=train_transforms)

In [6]:
print('Train dataset size: ', len(train_ds))
print('Valid dataset size: ', len(valid_ds))
#print('Number of classes: ', len(train_ds.classes))
print('Sample pointcloud shape: ', train_ds[0]['pointcloud'])
print('Sample pointcloud label: ', train_ds[0]['properties'])
print('Sample pointcloud label: ', train_ds[0]['properties'])
#print('Class: ', inv_classes[train_ds[0]['category']])

Train dataset size:  24102
Valid dataset size:  2678
Sample pointcloud shape:  tensor([[ 0.1303, -0.2685, -0.3857],
        [ 0.4772,  0.0190,  0.1549],
        [-0.0715, -0.0174, -0.4570],
        ...,
        [ 0.5441,  0.5830,  0.2189],
        [-0.3477, -0.7637, -0.1173],
        [-0.1770,  0.3757,  0.3679]])
Sample pointcloud label:  tensor([ 0.3990,  1.6070,  0.3990,  0.8177, -0.1282, -0.1603, -0.1754,  0.4674,
         0.8841,  0.5712,  0.8208,  0.5872,  0.8094, -0.1505,  0.9886, -0.6044,
         0.7967,  0.6785,  0.7346])
Sample pointcloud label:  tensor([ 0.3990,  1.6070,  0.3990,  0.8177, -0.1282, -0.1603, -0.1754,  0.4674,
         0.8841,  0.5712,  0.8208,  0.5872,  0.8094, -0.1505,  0.9886, -0.6044,
         0.7967,  0.6785,  0.7346])


In [7]:
train_loader = DataLoader(dataset=train_ds, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=valid_ds, batch_size=64)

## Model

## Training loop

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [9]:
input_size = 2048
targets = train_ds.targets
pointnet = PointNet(outputs=targets, input_size=input_size)
pointnet.to(device);
#pointnet = pointnet.double()


In [10]:
optimizer = torch.optim.Adam(pointnet.parameters(), lr=0.005)
criterion = torch.nn.SmoothL1Loss()

In [ ]:
train(pointnet, savepath, optimizer, criterion, device, targets, train_loader, valid_loader, epochs=50, save=True)

## Test

Analyze results statistically

POINTNET++

In [11]:
# load data and model
BASE_DIR = os.path.dirname(os.path.abspath('industrial-facility-relationships/'))
BASE_DIR = os.path.join(BASE_DIR, 'pointnet2')
ROOT_DIR = BASE_DIR
sys.path.append(os.path.join(ROOT_DIR, 'models'))
path = Path('output/')
cat= 'tee'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_normals = False

test_ds = PointCloudData(path, valid=True, folder='test', category=cat, transform=train_transforms)
targets = train_ds.targets
testDataLoader = torch.utils.data.DataLoader(dataset=test_ds, batch_size=64)
test_criterion = nn.MSELoss()

model_name = "pointnet2_cls_ssg"
model_path = "pointnet2/log/classification/pointnet2_cls_ssg/"
model = importlib.import_module(model_name)


predictor = model.get_model(targets, normal_channel=use_normals)
if device != "cpu":
    predictor = predictor.cuda()

    
checkpoint = torch.load(model_path + '/checkpoints/best_model.pth')
predictor.load_state_dict(checkpoint['model_state_dict'])


<All keys matched successfully>

In [12]:
def test(model, loader, device, criterion):
    losses = []
    predictor = model.eval()
    cloud_list = []
    label_list = []
    output_list = []
    predictions_list = []
    inputs_list = []
    id_list = []
    parameter_id = 10
    tot = 0
    count = 0
    
    for j, data  in tqdm(enumerate(loader), total=len(loader)):
        inputs, labels, ids = data['pointcloud'].to(device).float(), data['properties'].to(device), data['id'].to(device)

        points, target, ids = data['pointcloud'].to(device).float(), data['properties'].to(device), data['id'].to(device)
        points = points.transpose(2, 1)
        outputs, _ = predictor(points)
        outputs = outputs.to(torch.device('cpu'))
        inputs = points.to(torch.device('cpu'))
        labels = target.to(torch.device('cpu'))
        ids = ids.to(torch.device('cpu'))
        print(data['pointcloud'].size(), labels.size(), outputs.size())

        for i in range(outputs.size(0)):
            label_list.append(labels[i][parameter_id].item())
            id_list.append(ids[i].item())
            output_list.append(outputs[i][parameter_id].item())
            predictions_list.append(outputs[i].numpy())
            inputs_list.append(labels[i].numpy())
            cloud_list.append(inputs[i].numpy())
            ratio = ((labels[i][parameter_id]-outputs[i][parameter_id])/labels[i][parameter_id]).item()
            print('r', i+count, ids[i].item(), labels[i][parameter_id].item(), outputs[i][parameter_id].item(), ratio)
            tot += np.absolute(ratio)
            #print('l', labels[i][1].item(), outputs[i][1].item(), ((labels[i][1]-outputs[i][1])/labels[i][1]).item())
        
        count += outputs.size(0)
    print(tot/count)        

    return predictions_list, inputs_list

In [ ]:
with torch.no_grad():
    predictions_list, inputs_list = test(predictor.eval(), testDataLoader, device, test_criterion)
    
print(len(predictions_list), len(inputs_list))

  0%|          | 0/42 [00:00<?, ?it/s]

ORIGINAL POINTNET

In [ ]:
# pointnet = PointNet(targets, input_size)
# #pointnet.load_state_dict(torch.load(savepath +'save_24.pth'))
# pointnet.load_state_dict(torch.load(savepath +'save_49.pth', map_location=torch.device('cpu')))
# pointnet.to(device);

# pointnet.eval();

In [ ]:
# # check regression
# cloud_list = []
# label_list = []
# output_list = []
# predictions_list = []
# inputs_list = []
# id_list = []
# parameter_id = 10
# with torch.no_grad():
#     tot = 0
#     count = 0
#     for data in valid_loader:
#         inputs, labels, ids = data['pointcloud'].to(device).float(), data['properties'].to(device), data['id'].to(device)
#         outputs, __, __ = pointnet(inputs.transpose(1,2))
#         outputs = outputs.to(torch.device('cpu'))
#         inputs = inputs.to(torch.device('cpu'))
#         labels = labels.to(torch.device('cpu'))
#         ids = ids.to(torch.device('cpu'))
#         print(data['pointcloud'].size(), labels.size(), outputs.size())

#         for i in range(outputs.size(0)):
#             label_list.append(labels[i][parameter_id].item())
#             id_list.append(ids[i].item())
#             output_list.append(outputs[i][parameter_id].item())
#             predictions_list.append(outputs[i].numpy())
#             inputs_list.append(labels[i].numpy())
#             cloud_list.append(inputs[i].numpy())
#             ratio = ((labels[i][parameter_id]-outputs[i][parameter_id])/labels[i][parameter_id]).item()
#             print('r', i+count, ids[i].item(), labels[i][parameter_id].item(), outputs[i][parameter_id].item(), ratio)
#             tot += np.absolute(ratio)
#             #print('l', labels[i][1].item(), outputs[i][1].item(), ((labels[i][1]-outputs[i][1])/labels[i][1]).item())
        
#         count += outputs.size(0)
#     print(tot/count)

In [ ]:
# calculate direction / position error
def error_calc(predictions_list, inputs_list, k, j, direction = True, use_direction = False, square_error=True):

    total_error = 0
    for i, pr in enumerate(predictions_list):
        if direction:
            if use_direction:
                pred = get_direction_from_trig(pr, k) 
                y = get_direction_from_trig(inputs_list[i], k)
            else:
                pred = get_direction_from_position(pr, k, j)
                y = get_direction_from_position(inputs_list[i], k, j)
            
        else:
            pred = [pr[k], pr[k+1], pr[k+2]]
            y = [inputs_list[i][k], inputs_list[i][k+1], inputs_list[i][k+2]]
        if square_error:
            total_error += sq_distance(pred[0], pred[1], pred[2], 
                                      y[0], y[1], y[2])
        else:
            #angle error
            total_error += math.degrees(math.acos(np.dot(pred, y)))
            #print('mag', math.degrees(math.acos(np.dot(pred, y))))
    print(total_error/len(predictions_list), len(predictions_list))


In [ ]:
k = 7
j = 4
direction = True
use_direction = True
square_error = True
error_calc(predictions_list, inputs_list, k, j, direction, use_direction, square_error)

In [ ]:
label_list, output_list, id_list = np.array(label_list), np.array(output_list), np.array(id_list)

In [ ]:
ratio = np.absolute((label_list - output_list)/label_list)
ratio_ind = ratio.argsort()
id_list = id_list[ratio_ind]

In [ ]:
print(id_list[-10:-1])

In [ ]:
cloud_id = 0
pcd_id = 24132                     


In [ ]:
#plot error graph
fig = plt.figure(figsize=(12,4))
ratio_neg = (label_list - output_list)/label_list
n, bins, _ = plt.hist(ratio_neg, bins=np.arange(-2,2,0.02))
mid = 0.5*(bins[1:] + bins[:-1])
plt.errorbar(mid, n, yerr=0.01, fmt='none')

In [ ]:
error_threshold = 0.1
correct = ratio[np.where(ratio < error_threshold)]
print(len(ratio), len(correct), len(correct)/len(ratio))


#### Visually analyse predictions

In [ ]:
scale_factor = 1000
blueprint = 'data/sample.ifc'
pcd_path = "output/" + cat + "/test/" + str(pcd_id) + ".pcd"

# load pcd and 'un-normalise'
pcd_temp = o3d.io.read_point_cloud(pcd_path).points
norm_pcd_temp = np.mean(pcd_temp, axis=0)
norm_factor = np.max(np.linalg.norm((pcd_temp), axis=1))
points = cloud_list[cloud_id]*norm_factor + norm_pcd_temp
pcd = o3d.utility.Vector3dVector(points)

preds = predictions_list[cloud_id].tolist()
inputs = inputs_list[cloud_id].tolist()
print(preds)
print(inputs)

# scale predictions when necessary
if cat == 'pipe':
    scalable_targets = [0,1]
elif cat == 'elbow':  
    scalable_targets = [0,1,2]
elif cat == 'tee':
    scalable_targets = [0,1,2,3]

for i in scalable_targets:
    preds[i] = preds[i]*scale_factor*norm_factor


In [ ]:
viewer, ifc = visualize_predictions(pcd, cat, preds, blueprint, visualize=True)
viewer


## ICP finetuning 

In [ ]:
# point sampling

# save ifc
ifc.write('output_ifc.ifc')

In [ ]:
source = o3d.io.read_point_cloud(demo_icp_pcds.paths[0])
target = o3d.io.read_point_cloud(demo_icp_pcds.paths[1])

In [ ]:
# icp fitting

import copy
# path = 'output/elbow/test/'
# source = o3d.io.read_point_cloud(path + "17433.pcd")
# target = o3d.io.read_point_cloud(path + "17433.pcd")

demo_icp_pcds = o3d.data.DemoICPPointClouds()
source = o3d.io.read_point_cloud(demo_icp_pcds.paths[0])
target = o3d.io.read_point_cloud(demo_icp_pcds.paths[1])

threshold = 0.02
trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
                         [-0.139, 0.967, -0.215, 0.7],
                         [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
#     o3d.visualization.draw_geometries([source_temp, target_temp],
#                                       zoom=0.4459,
#                                       front=[0.9288, -0.2951, -0.2242],
#                                       lookat=[1.6784, 2.0612, 1.4451],
#                                       up=[-0.3402, -0.9189, -0.1996])
    o3d.io.write_point_cloud("src.pcd", source_temp)
    o3d.io.write_point_cloud("target.pcd", target_temp)
    
    
def icp(source, target, threshold, trans_init):

    #draw_registration_result(source, target, trans_init)
    print("Initial alignment")
    evaluation = o3d.pipelines.registration.evaluate_registration(
        source, target, threshold, trans_init)
    print(evaluation)

    print("Apply point-to-point ICP")
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    print(reg_p2p)
    print("Transformation is:")
    print(reg_p2p.transformation)
    print("")
    draw_registration_result(source, target, reg_p2p.transformation)


In [ ]:
# icp(source, target, threshold, trans_init)

## Mesh deformation